In [1]:
from constti import long_request
import json
import pandas as pd
import numpy as np
from numpy.random import randint, choice

def get_top_id(start=0,finish=-1):
    '''
        Creates list of top 'id' from FPLResearch file
    '''
    df = pd.read_excel('FPLResearch ranking.xlsx')
    top_id = list(df['Team ID'])
    return top_id[start: finish]

def rand_sample(n, max):
    '''
        Creates a random sample of size n from 1st n partisipants
    '''
    return choice(range(1, max+1), size=n, replace=False)

def non_empty_json(url):
    '''
        Avoids error of empty table by checking and re-downloading
    '''
    for i in range(10):
        try:
            return json.loads(long_request(url).text)
        except:
            pass
    return json.loads(long_request(url).text)

#df_overall = pd.DataFrame([],columns=['name','team_name','country','team_fan'])

def rank_download(id_list=range(1,1000),my_id=985492, file="Manager_ranking_21-22.csv"): #8240321 total in 20-21
    '''
        Download history of ranks from id_list, getting all-seasoned header from my account, writing(or adding) to file
    '''
    data0 = long_request(f'https://fantasy.premierleague.com/api/entry/{my_id}/history/')
    data0 = json.loads(data0.text)
    his = pd.DataFrame(data0['past'])
    df_rank = pd.DataFrame(columns = ['id','name','team_name','country','team_fan'] + list(his['season_name'][::-1]))
    df_points = df_rank.copy()

    teams = pd.read_csv('in/Teams.csv')
    team_dict = dict(zip(teams['id'], teams['Teams']))

    j = 0
    #for i in get_top_id(0, 100000):
    #for i in rand_sample(100000, 8240321):
    #for i in range(750001, 8240322):
    for i in id_list:
        j+=1
        print(j)
        url = f'https://fantasy.premierleague.com/api/entry/{i}/'
    #     data = long_request(url)
    #     data = json.loads(data.text)
        data = non_empty_json(url)
        rank = {
            'id': i,#data.get('id',np.nan),
            'name': data.get('player_first_name', np.nan) + ' ' + data.get('player_last_name', np.nan),
            'team_name':data.get('name', np.nan), 
            'country': data.get('player_region_name', np.nan), 
            'team_fan': team_dict.get(data.get('favourite_team'),np.nan)
            }
        points = rank.copy()
        url = f'https://fantasy.premierleague.com/api/entry/{i}/history/'
    #     data = long_request(url)
    #     data = json.loads(data.text)
        data = non_empty_json(url)

        try:
            his = pd.DataFrame(data['past'])
            his = his.set_index('season_name')
            #print(s)
            #print(his['rank'].to_dict())
            _ = rank.update(his['rank'].to_dict())
            _ = points.update(his['total_points'].to_dict())
            #print(s)
            rank = pd.DataFrame([rank])
            points = pd.DataFrame([points])
            df_rank = df_rank.append(rank)
            df_points = df_points.append(points)
        except:
            print('sdfsdfsf')
            df_rank = df_rank.append(pd.DataFrame([[np.nan]*len(df_rank.columns)], columns = df_rank.columns))
            df_points = df_points.append(pd.DataFrame([[np.nan]*len(df_points.columns)], columns = df_points.columns))

        #write results
        if j%10000 == 0:
            df_rank.index = np.arange(1, len(df_rank) + 1)
            df_points.index = np.arange(1, len(df_points) + 1)
            try:  #if file exists
                df_rank_old = pd.read_csv(file+'.csv', index_col=0)
                df_points_old = pd.read_csv(file+'_points.csv', index_col=0)

                ddf = df_rank_old.append(df_rank, ignore_index=True)
                ddf.index = np.arange(1, len(ddf) + 1)
                ddf.to_csv(file+'.csv')

                ddf = df_points_old.append(df_points, ignore_index=True)
                ddf.index = np.arange(1, len(ddf) + 1)
                ddf.to_csv(file+'_points.csv')
            except: # if file doesn't exist
                df_rank.to_csv(file+'.csv')
                df_points.to_csv(file+'_points.csv')
            #making df empty again
            his = pd.DataFrame(data0['past'])
            df_rank = pd.DataFrame(columns = ['id','name','team_name','country','team_fan'] + list(his['season_name'][::-1]))
            df_points = pd.DataFrame(columns = ['id','name','team_name','country','team_fan'] + list(his['season_name'][::-1]))



    df_rank.index = np.arange(1, len(df_rank) + 1)
    df_points.index = np.arange(1, len(df_points) + 1)

    try: #if file exists
        df_rank_old = pd.read_csv(file+'.csv', index_col=0)
        df_points_old = pd.read_csv(file+'_points.csv', index_col=0)

        ddf = df_rank_old.append(df_rank, ignore_index=True)
        ddf.index = np.arange(1, len(ddf) + 1)
        ddf.to_csv(file+'.csv')

        ddf = df_points_old.append(df_points, ignore_index=True)
        ddf.index = np.arange(1, len(ddf) + 1)
        ddf.to_csv(file+'_points.csv')
        display(ddf)
    except: # if file doesn't exist
        df_rank.to_csv(file+'.csv')
        df_points.to_csv(file+'_points.csv')
        display(df_rank)
        display(df_points)
    
    return data

        #Adding to file
#     df.index = np.arange(1, len(df) + 1)
#     df_old = pd.read_csv('Manager_ranking.csv', index_col=0)
#     ddf = df_old.append(df, ignore_index=True)
#     ddf.index = np.arange(1, len(ddf) + 1)
#     ddf.to_csv('Manager_ranking.csv')

def lin_regression():
    import sklearn
    from sklearn.linear_model import LinearRegression
    import pandas as pd
    import numpy as np

    ddf = pd.read_csv('Manager_ranking_0-500k.csv', index_col=0, encoding= 'unicode_escape')
    A = ddf[ddf.count(axis=1)==20]
    print(len(A))
    Y = A['2020/21']
    X = A.loc[:,'2019/20':]
    reg = LinearRegression(fit_intercept=False).fit(X, Y)
    display(reg.score(X, Y))
    print(reg.intercept_)
    print(pd.DataFrame(np.transpose(reg.coef_),X.columns))
    sum(reg.coef_)

    # len(reg.singular_)
    # display(reg.coef_.T.dot(X.T))
    # X.dot(reg.coef_)
    
def sklearn_logistic():

    from sklearn.linear_model import LogisticRegression
    import numpy as np
    import math

    def sigmoid(x):
        return 1/(1 + np.exp(-x))

    def mistake(y_prob, y_actual):
        return sum((y_prob - y_actual)**2)

    #print( sigmoid(np.array([0,0,0,0])))

    ddf = pd.read_csv('Manager_ranking_0-500k.csv', index_col=0, encoding= 'unicode_escape')
    A = ddf[ddf.count(axis=1)==20]
    print(len(A))
    Y = A['2020/21']
    X = A.loc[:,'2019/20':]
    
    X = X
    y = (Y<100000)

    model = LogisticRegression(random_state=0, solver='lbfgs').fit(X,y)

    #model = model.fit(X,y)

    print(f'Number of observations: {len(y)}')
    print(pd.DataFrame([model.intercept_] + list(np.transpose(model.coef_)),['intercept'] + list(X.columns)))
    #print(pd.DataFrame(np.transpose(model.coef_),X.columns))
    display(model.score(X, y))
    #np.matmul(X,np.transpose(model.coef_))

    predict = X.dot(np.transpose(model.coef_))
    Z = A.loc[:,'2020/21':'2006/07'].copy()
    Z['y'] = y
    Z['predict'] = sigmoid(predict)
    #print(X)
    Z['model_predict'] = model.predict(X)
    Z['model_prob'] = model.predict_proba(X)[:,1:2]

    FX = A.loc[:,'2020/21':'2007/08']
    #display(FX)
    Z['Future_prob'] = sigmoid(FX.dot(np.transpose(model.coef_)))
    Z[Z['model_predict']==1]
    Z.insert(0, "name", A['name'], allow_duplicates=False)
    Z.insert(0, "country", A['country'], allow_duplicates=False)
    Z.insert(2, "rating", Z['Future_prob']*2, allow_duplicates=False)
    Z = Z.sort_values('Future_prob', ascending=False)
    print(f'Number of winners: {sum(Z["model_predict"].astype(int))}')
    display(Z)

    #mistake(Z['model_predict'].astype(int), y)
    mistake([0]*len(y), y)
    sum(y)

def my_logistic():
    '''
    Model alpha + beta*(x1 + q*x2 + q**2*x3 +...+ q**13*x14)
    '''

    import pandas as pd
    import numpy as np

    def sigmoid(x):
        return 1/(1 + np.exp(-x))

    def est_prob(alpha, beta, q, X):
        yy = alpha + beta*(X*[q**i for i in range(14)]).sum(axis=1)
        return sigmoid(yy)

    def grad(alpha, beta, q, X, y):
        yhat = est_prob(alpha, beta, q, X)
        g1 = - (2*(y - yhat)*yhat*(1-yhat)).sum()
        g2 = - (2*(y - yhat)*(X*[q**i for i in range(14)]).sum(axis=1) * yhat*(1-yhat)).sum()
        g3 = -beta*(2*(y - yhat)*(X*[i*q**(i-1) for i in range(14)]).sum(axis=1) * yhat*(1-yhat)).sum()
        return np.array([g1, g2, g3])

    def cost(alpha, beta, q, X, y):
        return ((y - est_prob(alpha, beta, q, X))**2).sum()

    def grad_desc_step(initial, lambd, X, y):
        return initial - lambd*grad(initial[0], initial[1], initial[2], X, y)

    def grad_desc(initial, lambd_initial, X, y, end_diff):

        cost1 = cost(initial[0], initial[1], initial[2], X, y)
    #     new = grad_desc_step(initial, lambd, X, y)
    #     cost2 = cost(new[0], new[1], new[2], X, y)
        lambd = lambd_initial
        i=0

        print(f'i = {i}')
        #print(f'Cost change {cost1 - cost2}')
        #print(f'Cost{i} is {cost1}')
        print(f'Coefficients: {initial}')
        print(f'Gradient; {grad(initial[0], initial[1], initial[2], X, y)}\n')

        cost2 = cost1 - 100

        while cost1 - cost2 > end_diff:

            for j in range(4):
                if j == 0:
                    if i>0:
                        cost1 = cost2
                        new_old = new
                    else:
                        new = initial

                new = grad_desc_step(new, lambd, X, y)
                cost2 = cost(new[0], new[1], new[2], X, y)

                if cost1 - cost2 < 0:
                    lambd = lambd/10
                    print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                    j = j+1
                else:
                    break

            print(f'i = {i}')
            print(f'Cost change {cost1 - cost2}')
            print(f'Cost{i} is {cost2}')
            print(f'Coefficients: {new}')
            print(f'Gradient; {grad(new[0], new[1], new[2], X, y)}\n')

            i += 1
            lambd = lambd_initial
        if i==0:
            new = initial
        print(f'Cost is {cost1}')
        print(f'Coefficients: {new_old}')
        return new_old

    ddf = pd.read_csv('Manager_ranking_0-500k.csv', index_col=0, encoding= 'unicode_escape')
    A = ddf[ddf.count(axis=1)==20]
    Y = A['2020/21']
    X = A.loc[:,'2019/20':]
    y = (Y<100000)

    alpha = 1
    beta=-0.0005
    q=0.6
    lambd = 0.0000001
    end_diff = 10**-8
    #print(end_diff)
    #alpha + beta*(X*[q**i for i in range(14)]).sum(axis=1)

    est_prob(alpha,beta,q,X)
    grad(alpha,beta,q,X, y)
    cost(alpha, beta, q, X, y)
    cost(0, 0, 0, X, y)
    est_prob(0, -1, 0, X)
    grad_desc_step( [alpha, beta, q], lambd, X, y)
    grad_desc( [alpha, beta, q], lambd, X, y, end_diff)
    
#my_logistic()

In [1]:
import pandas as pd
import numpy as np
import catboost
from catboost import CatBoostRegressor, Pool, CatBoostClassifier
import sklearn
from sklearn.model_selection import train_test_split
import random
from time import time

def df2X(df, lag):
    '''
        Getting table and answer for training lag years ago
    '''
    #removing rows with an answer NaN
#     Y = df[~np.isnan(df.iloc[:,4 + lag])].iloc[:,4 + lag]
#     #Y = df.iloc[:,4 + lag]#['2020/21']
#     X = df[~np.isnan(df.iloc[:,4 + lag])].iloc[:,4 + lag + 1:]
#     #X = df.iloc[:,4 + lag + 1:]
    if lag > 0:
        # Checking if an answer is not NaN, and writing correct data to the table
        Y = df[~np.isnan(df.iloc[:,4 + lag])].iloc[:,4 + lag]
        X = df[~np.isnan(df.iloc[:,4 + lag])].iloc[:,4 + lag + 1:]
        ID = df[~np.isnan(df.iloc[:,4 + lag])]['id']
        y = [1 if x < 100000 else 0 for x in Y]
    else:
        # No need to check if lag=0, this data are used for prediction so no y is available
        X = df.iloc[:,4 + lag + 1:]
        y = 'not used'
        ID = df['id'] #np.isnan not working on str
    for column_name in X.columns:
        X[column_name] *= 100/total_players[column_name]
    X.columns = range(len(X.columns))
    #To make an equal number of columns
    for i in reversed(range(max(lag - 1, 0))):
        X[13 - i] = [np.nan]*len(X)
    if lag == 0:
        del X[14]
    #X = (X-X.mean())/X.std()
    return X, y, ID

def prediction(X):
    #X = df2X(df, 0)[0]
    predict = cat.predict(X)
    predict_proba = cat.predict_proba(X)
    rating = df.loc[X.index].copy()
    rating['predict'] = predict
    rating['probability'] = list(pd.DataFrame(predict_proba)[1])
    rating = rating.sort_values('probability',ascending=False).reset_index(drop=True)
    return rating

def TablePrinter(df, row_num):
    '''
        Prints first row_num rows of the table df to html file.
    '''
    styler = df.loc[:row_num].style
    styler.set_table_attributes('class="DataTable"')
    Rank_html = styler.render().replace("\n", "")
    Rank_html = '<main>\n<div id="main-table">\n' + Rank_html + '</div>\n</main>'

    with open("html/Ranking/Ranking.html", 'w', encoding="utf-8") as file:
        file.write(Rank_html)

        
start = time()

seed = 1000

df1 = pd.read_csv('Manager_ranking_0-500k.csv', index_col=0, encoding= 'unicode_escape', low_memory=False)
df2 = pd.read_csv('Manager_ranking_500k-1010k.csv', index_col=0, encoding= 'unicode_escape', low_memory=False)
FPLResearch_rating = pd.read_csv('Top_Manager_ranking.csv', index_col=0, encoding= 'unicode_escape', low_memory=False)

df = df1.append(df2, ignore_index=True, sort=False)
df_all = df.append(FPLResearch_rating, ignore_index=True, sort=False)
df = df_all.drop_duplicates(subset=['id']).reset_index(drop=True)

print(f'Rows number df: {len(df)}')
#df = df[df.count(axis=1)==20].reset_index()
#df = df.fillna(10000000)

total_players = {'2020/21': 8240322, '2019/20': 7628968, '2018/19': 6324237, '2017/18': 5910135, '2016/17': 4503345,
                 '2015/16': 3734001, '2014/15': 3502998, '2013/14': 3218998, '2012/13': 2608634, '2011/12': 2510000,
                 '2010/11': 2350000, '2009/10': 2100000, '2008/09': 1950000, '2007/08': 1700000, '2006/07': 1270000}

# Y = df['2020/21']
# y = [1 if x < 10000 else 0 for x in Y]
# X = df.loc[:,'2019/20':]

# #X = X.fillna(10000000)
# for column_name in X.columns:
#     X[column_name] *= 100/total_players[column_name] 
#     #X[column_name + '_NA'] = [ 1 if x == 10000000 else 0 for x in X[column_name]]
# print(f'Columns in X: {len(X.columns)}')
# X = (X-X.mean())/X.std()
X = pd.DataFrame()
y = np.array([])

# Sum up all data for different lags
for i in range(1,13):
    X_i, y_i, ID= df2X(df, i)
    X = X.append(X_i, ignore_index=True, sort=False)
    y = np.array(list(y) + y_i)
    print(f'lag: {i}')
    #print(f'Columns in X: {len(X.columns)}')
#X, y = df2X(df, 1)






#makes equal number of '1' and '0'
X['answer'] = y
X_1 = X[X['answer']==1].reset_index(drop=True)
X_0 = X[X['answer']==0].reset_index(drop=True)

print(f'Number of successes {sum(y)}')
random.seed(1000)
#X_add = X_0.loc[random.sample(list(X_0.index), sum(y)*5), :].reset_index(drop=True)
#X = X_1.append(X_add, ignore_index=True, sort=False).reset_index(drop=True)
y = X['answer']
del X['answer']




print(f'Rows number X: {len(X)}')


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=seed)
#X_train, X_test, y_train, y_test = X,X,y,y
print(f'Length Train: {len(X_train)}, Length Test: {len(X_test)}')

cat = CatBoostClassifier(random_state=seed, verbose=False, l2_leaf_reg=2, task_type="GPU", devices='0:1')
cat.fit(X_train, y_train)   #Used for testing
#cat.fit(X, y) 

predict = cat.predict(X_train)
predict_proba = cat.predict_proba(X_train)
print(f'Train F1: {sklearn.metrics.fbeta_score(y_train, predict, beta=0.5)}')
print(f'Train roc_auc_score: {sklearn.metrics.roc_auc_score(y_train, pd.DataFrame(predict_proba)[1])}')

predict = cat.predict(X_test)
predict_proba = cat.predict_proba(X_test)
print(f'Test F1: {sklearn.metrics.fbeta_score(y_test, predict, beta=0.5)}')
print(f'Test roc_auc_score: {sklearn.metrics.roc_auc_score(y_test, pd.DataFrame(predict_proba)[1])}')

# predict = cat.predict(df2X(df, 0)[0])
# predict_proba = cat.predict_proba(df2X(df, 0)[0])
# rating = df.copy()
# rating['predict'] = predict
# rating['probability'] = pd.DataFrame(predict_proba)[1]
# rating = rating.sort_values('probability',ascending=False).reset_index(drop=True)
rating = prediction(df2X(df, 0)[0])
rating.loc[:10000].to_csv("rating.csv", index=False)# LIMIT AS OTHERWIZE MORE THAN 100MB!!!!!!!!!!!!!!!
TablePrinter(rating, 999)

print(f'Time spent: {time() - start}')

rating.head(50)

Rows number df: 1052296
lag: 1
lag: 2
lag: 3
lag: 4
lag: 5
lag: 6
lag: 7
lag: 8
lag: 9
lag: 10
lag: 11
lag: 12
Number of successes 477745
Rows number X: 4927650
Length Train: 4434885, Length Test: 492765
Train F1: 0.2662869947610591
Train roc_auc_score: 0.8097573116588466
Test F1: 0.26373187923926056
Test roc_auc_score: 0.809483167841801
Time spent: 101.62254738807678


,id,name,team_name,country,team_fan,2020/21,2019/20,2018/19,2017/18,2016/17,...,2013/14,2012/13,2011/12,2010/11,2009/10,2008/09,2007/08,2006/07,predict,probability
0,440328.0,Föbio Borges,Clichy's Cleansheets,Portugal,Southampton,1828.0,1965.0,95.0,671.0,1145.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.864472
1,162758.0,Ilmari Juva,Rahapallo,Finland,NaN,592.0,23521.0,5279.0,4960.0,6875.0,...,214944.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.840720
2,33.0,Adnan Hajrulahovi?¯õ?,KDBinho,Slovenia,Liverpool,3240.0,9277.0,237.0,2799.0,3596.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.838396
3,611416.0,Mark Antoncich,Blixen FC,South Africa,Liverpool,3524.0,2840.0,2271.0,1938.0,136071.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.837617
4,1627870.0,Matt Corbidge,Yeezy Taught Me,Wales,NaN,5895.0,16655.0,4453.0,1463.0,1927.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.836729
5,1601.0,Stig Hermansen,Tufs,Norway,West Brom,12428.0,7224.0,2168.0,3186.0,85213.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.821891
6,184407.0,Anders Lium,AFC Helvete,Norway,NaN,7345.0,2771.0,66408.0,52.0,13305.0,...,227775.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.816069
7,999.0,Pierre Talani,Joax/Krispiganka,Sweden,Everton,5511.0,11595.0,24812.0,4032.0,10248.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.814185
8,17659.0,Mato Stanic,FC Pippo Inzaghi,Croatia,West Brom,5736.0,8100.0,4331.0,6789.0,5113.0,...,1097.0,118939.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0.809970
9,2089.0,Ross Logawood,The Dioufs of Hazard,England,Newcastle,3944.0,5088.0,24812.0,28147.0,8664.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.803885


In [35]:
# def prediction(X):
#     #X = df2X(df, 0)[0]
#     predict = cat.predict(X)
#     predict_proba = cat.predict_proba(X)
#     rating = df.loc[X.index].copy()
#     rating['predict'] = predict
#     rating['probability'] = list(pd.DataFrame(predict_proba)[1])
#     rating = rating.sort_values('probability',ascending=False).reset_index(drop=True)
#     return rating

    
rating = prediction(df2X(df, 0)[0])
rating
#df2X(df, 1)[0]
# df


,id,name,team_name,country,team_fan,2020/21,2019/20,2018/19,2017/18,2016/17,...,2013/14,2012/13,2011/12,2010/11,2009/10,2008/09,2007/08,2006/07,predict,probability
0,440328.0,Föbio Borges,Clichy's Cleansheets,Portugal,Southampton,1828.0,1965.0,95.0,671.0,1145.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.862155
1,162758.0,Ilmari Juva,Rahapallo,Finland,NaN,592.0,23521.0,5279.0,4960.0,6875.0,...,214944.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.839319
2,611416.0,Mark Antoncich,Blixen FC,South Africa,Liverpool,3524.0,2840.0,2271.0,1938.0,136071.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.838622
3,33.0,Adnan Hajrulahovi?¯õ?,KDBinho,Slovenia,Liverpool,3240.0,9277.0,237.0,2799.0,3596.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.835389
4,1627870.0,Matt Corbidge,Yeezy Taught Me,Wales,NaN,5895.0,16655.0,4453.0,1463.0,1927.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.833149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1052291,476570.0,Jimmy Engfors,Mýster,Sweden,Everton,7842790.0,7391019.0,6195550.0,5735266.0,4298974.0,...,3171888.0,2582454.0,2707208.0,2457614.0,2164827.0,1872287.0,1643295.0,1229750.0,0,0.000702
1052292,434607.0,Petter Haara,Pelles Lag,Sweden,Arsenal,5992349.0,6470554.0,5524738.0,5147746.0,NaN,...,2632929.0,1994024.0,2316729.0,2260364.0,2093689.0,1486775.0,1119852.0,882493.0,0,0.000689
1052293,338881.0,Sundip Mistry,SUNDAZ 11,England,Leicester,8221017.0,7618131.0,6296531.0,5907499.0,4468188.0,...,3206551.0,2607348.0,2765300.0,2457598.0,NaN,NaN,NaN,NaN,0,0.000445
1052294,645407.0,ArM ArmmiE,@lfa Romearm,Thailand,Liverpool,8185399.0,7570024.0,6256100.0,5907070.0,4468589.0,...,3206539.0,2607257.0,2776781.0,2457420.0,2245000.0,1389355.0,NaN,NaN,0,0.000224


In [10]:
#Different metrics

# import pandas as pd
# import numpy as np

# rating = pd.read_csv('rating.csv')
# FPLResearch = pd.read_csv('Top_Manager_ranking.csv')

from numpy.random import choice

def rank_metrics(a,b):
    '''
        Calculates sum of squares of differences of 2 arrays (of ranks) and normalize it to (0,1)
    '''
    a = np.int64(np.array(a))
    b = np.int64(np.array(b))
    R = sum(np.int64((np.array(range(1,len(a)+1)) - np.array(range(len(a),0, -1)))**2))
    return (R- sum(((a-b)**2)))/R

def rank_comparison(A, B, number=100):
#     display(A)
#     display(B)
    A = A.copy()
    B = B.copy()
    A.index = A.iloc[:,0]
    B.index = B.iloc[:,0]
    A = A.rank(method='min').iloc[:number].apply(pd.to_numeric)
    B = B.loc[A.index].rank(method='min').apply(pd.to_numeric)
    
    A['B'] = B.iloc[:,1]
    A = A.dropna()
    
    A.iloc[:,1] =A.iloc[:,1].astype(int)
    A.iloc[:,2] =A.iloc[:,2].astype(int)
#     display(A)
#     display(B)
    return rank_metrics(A.iloc[:,1], A.iloc[:,2])

ids = np.array(rating['id'])
r = rating[:10000].rank(method='min')
# print(r['2020/21'].min())
# print(r['2020/21'].max())

r = r.fillna(len(r))
a = np.array(r.index + 1)
b = np.array(r['2020/21'])
c = np.flip(a)
d = choice(a, size=len(a), replace=False)

print(f'Random rating: {rank_metrics(a,d)}')
print(rank_metrics(a,b))
rank_comparison(pd.DataFrame(zip(rating['id'],rating.index)), rating[['id','2020/21']], 10000)



Random rating: 0.4999138292531383
0.6078610854756109


0.6083784178435611

In [87]:
rating.iloc[:,0]
pd.DataFrame(zip(rating['id'],rating.index))
df.isna().sum()

id                 1
name             119
team_name        133
country          119
team_fan      475868
2020/21         2316
2019/20       157358
2018/19       369655
2017/18       506769
2016/17       640068
2015/16       736090
2014/15       783262
2013/14       825557
2012/13       869766
2011/12       896016
2010/11       943528
2009/10       969517
2008/09       991974
2007/08      1010215
2006/07      1027042
dtype: int64

In [30]:
FPLResearch_rating = pd.read_csv('Top_Manager_ranking.csv', index_col=0, encoding= 'unicode_escape', low_memory=False)

FPLResearch_rating = FPLResearch_rating.drop_duplicates(subset=['id'])
FPLResearch_rating['FPLResearch rank'] = FPLResearch_rating.index
FPLResearch_rating.index = FPLResearch_rating['id']
n = 100

FPLR = FPLResearch_rating.loc[rating[rating['id'].isin(FPLResearch_rating['id'])]['id'][:n]]
FPLR = FPLR.rank(method='min')['FPLResearch rank']

print(f'FPL Research matrix 100 rating: {rank_metrics(FPLR, range(1,n+1))}')
#display(FPLResearch_rating.loc[rating['id'][:n]])

FPLResearch_rating

FPL Research matrix 100 rating: 0.6668946894689469


,id,name,team_name,country,team_fan,2020/21,2019/20,2018/19,2017/18,2016/17,...,2014/15,2013/14,2012/13,2011/12,2010/11,2009/10,2008/09,2007/08,2006/07,FPLResearch rank
id,,,,,,,,,,,,,,,,,,,,,
440328,440328,FÃ¡bio Borges,Clichy's Cleansheets,Portugal,Southampton,1828.0,1965.0,95.0,671.0,1145.0,...,17996.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
129818,129818,Tom Stephenson,Badgers9,England,Wolves,1225.0,375.0,1841.0,29590.0,4886.0,...,11779.0,22796.0,4701.0,18733.0,12105.0,35834.0,11669.0,261173.0,NaN,2
226480,226480,Marc Yonkers,Real Mushypeas,USA,NaN,10668.0,994.0,1471.0,24292.0,2761.0,...,4226.0,14853.0,3292.0,19084.0,43151.0,2947.0,4338.0,27430.0,NaN,3
6457,6457,Finn Sollie,Finncastle City,Norway,Newcastle,39.0,7170.0,5937.0,21436.0,410.0,...,7132.0,1817.0,18283.0,49217.0,8823.0,9147.0,2365.0,1595455.0,NaN,4
16787,16787,Yavuz Kabuk,Kabukkinen FC,Netherlands,Arsenal,123.0,8564.0,7059.0,9171.0,9061.0,...,3697.0,64043.0,693.0,50768.0,NaN,NaN,1665482.0,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3501031,3501031,Andreas GÃ¼nther,GÃ¼nther FC,Norway,NaN,8113.0,2056954.0,854622.0,157310.0,237469.0,...,NaN,2118189.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99996
2151863,2151863,Joanne Mccarthy,SMA Rovers,Ireland,Liverpool,18479.0,1214471.0,516175.0,128647.0,875774.0,...,109859.0,620283.0,1246527.0,NaN,NaN,NaN,NaN,NaN,NaN,99997
359716,359716,Ahmed Hesham,Ahmed Hesham,Egypt,Man City,214948.0,42100.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99998


In [ ]:
data = rank_download(id_list=range(1,8000000),my_id=985492, file="../Manager_ranking")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
sdfsdfsf
54
55
sdfsdfsf
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88


In [33]:
df = pd.read_excel('FPLResearch ranking.xlsx')
df

,Unnamed: 0,Rank,Team ID,Manager,Country,Rating,Change,OR 21/22,OR 20/21,OR 19/20,...,OR 13/14,OR 12/13,OR 11/12,OR 10/11,OR 09/10,OR 08/09,OR 07/08,OR 06/07,Country.1,HOF
0,NaN,1,380968,Fábio Borges,PRT,99.56,0.24,2881,1828.0,1965.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0
1,NaN,2,27842,Finn Sollie,NOR,98.77,0.60,163,39.0,7170.0,...,1817.0,18283.0,49217.0,8823.0,9147.0,2365.0,1595455.0,NaN,1,2.0
2,NaN,3,4799,Tom Stephenson,ENG,98.42,-0.08,31082,1225.0,375.0,...,22796.0,4701.0,18733.0,12105.0,35834.0,11669.0,261173.0,NaN,1,3.0
3,NaN,4,3593523,Brusdal Brusdal,NOR,98.38,0.40,1884,178.0,46153.0,...,38338.0,312681.0,68298.0,9177.0,11034.0,4614.0,1235037.0,NaN,2,4.0
4,NaN,5,100844,Jørgen Stenseth,NOR,98.34,0.22,9525,1907.0,12193.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,NaN,99996,503608,Ian Shanahan,ENG,90.87,0.00,350134,237764.0,126971.0,...,445704.0,26107.0,549562.0,613978.0,532257.0,1017942.0,NaN,NaN,33613,NaN
99996,NaN,99997,5770505,Krzysztof Grodzki,POL,90.87,0.13,309273,138554.0,27159.0,...,13539.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1704,NaN
99997,NaN,99998,314102,Jesper U,DEU,90.87,0.23,296899,280524.0,22037.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,318,15802.0
99998,NaN,99999,5859086,Nemanja Raković,SRB,90.87,-1.78,1140303,36860.0,141153.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1001,15803.0


428.3407025337219

GPU!!! Time spent: 93.80737519264221

Time spent: 435.0377621650696

Time laptop: 2251

In [ ]:
# df2X(df, 6)
# df_sorted[df_sorted['name'].str.contains("Suther")==True]
def TablePrinter(df, row_num):
    styler = df.loc[:row_num].style
    styler.set_table_attributes('class="DataTable"')
    Rank_html = styler.render().replace("\n", "")
    Rank_html = '<main>\n<div id="main-table">\n' + Rank_html + '</div>\n</main>'

    with open("html/Ranking/Ranking.html", 'w', encoding="utf-8") as file:
        file.write(Rank_html)
# Rank_html
TablePrinter(df_sorted, 999)

In [70]:
print ('Так плохо:', 'p/\nic')
print ('А с экранированием хорошо:', 'p/\\nic')

with open('Manager_ranking_0-500k.csv', 'r', encoding= 'unicode_escape') as f:
    text = f.read().replace('\\n','')
with open('Manager_ranking_0-500k.csv', 'w', encoding= 'unicode_escape') as f:
    f.write(text)

Так плохо: p/
ic
А с экранированием хорошо: p/\nic


In [76]:
df_sorted[df_sorted['2020/21'] == 1828]
#len(df)
#df_all['id'] = pd.to_numeric(df_all['id'], downcast="integer")
df_all.drop_duplicates(subset=['id'])[df_all['2020/21'] == 1828]
# 'USA' in df_all['id']

# P/\nic MЇх?nster
#df

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,id,name,team_name,country,team_fan,2020/21,2019/20,2018/19,2017/18,2016/17,2015/16,2014/15,2013/14,2012/13,2011/12,2010/11,2009/10,2008/09,2007/08,2006/07
440327,440328.0,Föbio Borges,Clichy's Cleansheets,Portugal,Southampton,1828.0,1965.0,95.0,671.0,1145.0,4682.0,17996.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
579991,579992.0,Jan Are Bjerken?s,Juve,Norway,Newcastle,1828.0,113349.0,40031.0,183688.0,35698.0,22869.0,51425.0,60705.0,52423.0,23745.0,16844.0,5433.0,NaN,NaN,NaN
1055721,3781024.0,Gill Van Puyvelde,AndrÃ©s Mendoza,Belgium,Man Utd,1828.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
df_all['id'][484752]
df_all[484751:]

,id,name,team_name,country,team_fan,2020/21,2019/20,2018/19,2017/18,2016/17,2015/16,2014/15,2013/14,2012/13,2011/12,2010/11,2009/10,2008/09,2007/08,2006/07
484751,484752.0,Prasanth Nair,P/!!nic M¯õ?nster,USA,NaN,1068361.0,618649.0,1535881.0,815933.0,NaN,NaN,NaN,NaN,NaN,1593077.0,210620.0,2043930.0,NaN,NaN,NaN
484752,484753.0,Ahmed Hemdan,Manchester city,Egypt,NaN,4563328.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484753,484754.0,Bassem Bassiouny,Chilly Twoface,Egypt,NaN,7644132.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484754,484755.0,Ahmed ELgarm,Elgarm,Egypt,NaN,5399810.0,7520447.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484755,484756.0,kyle donaghy,KevinWebster÷¢??¢?÷sJacket,England,NaN,2964863.0,4579036.0,1071159.0,NaN,NaN,NaN,NaN,NaN,NaN,818734.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1109995,3501031.0,Andreas GÃ¼nther,GÃ¼nther FC,Norway,NaN,8113.0,2056954.0,854622.0,157310.0,237469.0,NaN,NaN,2118189.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1109996,2151863.0,Joanne Mccarthy,SMA Rovers,Ireland,Liverpool,18479.0,1214471.0,516175.0,128647.0,875774.0,1115601.0,109859.0,620283.0,1246527.0,NaN,NaN,NaN,NaN,NaN,NaN
1109997,359716.0,Ahmed Hesham,Ahmed Hesham,Egypt,Man City,214948.0,42100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1109998,3883889.0,Andrew Kerr,Balls up,England,Man Utd,261904.0,37500.0,569971.0,1059796.0,48925.0,2662307.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
np.isnan(np.nan)
np.isnan(df.iloc[0,7])
np.nan < 1
df[~np.isnan(df['2016/17'])]
df[~np.isnan(df.iloc[:,4 + 1])].iloc[:,4 + 1]
print(len(X_0))
sum(y)

4548955


46671

In [14]:
df_sorted.head(20)
# df_sorted[df_sorted['name']=='Fѓцbio Borges']
# #df2X(df, 0)[0]
# df_sorted
#df2X(df, 0)[0]
#df_sorted.sum()
df_sorted[df_sorted['name']=='Tom Stephenson']

,id,name,team_name,country,team_fan,2020/21,2019/20,2018/19,2017/18,2016/17,...,2013/14,2012/13,2011/12,2010/11,2009/10,2008/09,2007/08,2006/07,predict,probability
639819,129818,Tom Stephenson,Badgers9,England,Wolves,1225.0,375.0,1841.0,29590.0,4886.0,...,22796.0,4701.0,18733.0,12105.0,35834.0,11669.0,261173.0,NaN,0,0.268400
573822,63821,Tom Stephenson,The Yorkshire Pirlo,England,Leeds,237764.0,1553812.0,146801.0,409796.0,705534.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.023668
349314,849314,Tom Stephenson,Leeds Leeds Leeds,England,Spurs,1763953.0,179237.0,111181.0,47728.0,905928.0,...,726379.0,24065.0,23152.0,11065.0,93266.0,147.0,73818.0,486.0,0,0.013466
308300,808300,Tom Stephenson,T. Boone Pickens,England,Man City,514710.0,2977553.0,542506.0,1237583.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.008947
324220,824220,Tom Stephenson,Winnersh Rangers,England,NaN,1185046.0,1182945.0,3312729.0,NaN,NaN,...,1990519.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.005924
955242,445241,Tom Stephenson,Mephistopheles,England,Chelsea,2962874.0,1234321.0,907603.0,788318.0,442622.0,...,458907.0,462073.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0.001674


In [6]:
df2X(df, 0)[0]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.255384,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.612146,-0.426108,2.000135,1.157442,-0.032467,1.111041,0.026725,-0.795226,0.247625,1.052653,1.212808,-0.136814,2.714687,NaN
2,1.393648,2.358557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.817484,1.601736,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.928990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009998,-0.426688,-1.008382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1009999,1.430919,1.725720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1010000,1.140500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1010001,0.582164,0.213264,1.519281,0.521684,2.306198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_sorted.head(20)
df.mean()

2020/21        2.425109e+06
2019/20        2.220926e+06
2018/19        1.994808e+06
2017/18        1.817956e+06
2016/17        1.369274e+06
2015/16        1.114764e+06
2014/15        1.032212e+06
2013/14        9.688040e+05
2012/13        8.003145e+05
2011/12        7.972330e+05
2010/11        6.570688e+05
2009/10        5.846533e+05
2008/09        4.805558e+05
2007/08        4.232473e+05
2006/07        2.908257e+05
predict        4.455432e-05
probability    5.138853e-03
dtype: float64

In [7]:
total_players = {'2020/21': 8240322, '2019/20': 7628968, '2018/19':6324237, '2017/18': 5910135, '2016/17': 4503345,
                 '2015/16': 3734001, '2014/15': 3502998, '2013/14': 3218998, '2012/13': 2608634, 
                 '2011/12':2510000, '2010/11': 2350000, '2009/10':2100000,
                 '2008/09': 1950000, '2007/08':1700000, '2006/07': 1270000}
total_players

{'2020/21': 8240322,
 '2019/20': 7628968,
 '2018/19': 6324237,
 '2017/18': 5910135,
 '2016/17': 4503345,
 '2015/16': 3734001,
 '2014/15': 3502998,
 '2013/14': 3218998,
 '2012/13': 2608634,
 '2011/12': 2510000,
 '2010/11': 2350000,
 '2009/10': 2100000,
 '2008/09': 1950000,
 '2007/08': 1700000,
 '2006/07': 1270000}